<a href="https://colab.research.google.com/github/anubhavgupta1/Deep-Learning-Using-TensorFlow/blob/master/1_Hands_on_Tensorlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install tensorflow==1.10.0


In [45]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

In [46]:
print(tf.__version__)

1.10.0


In [47]:
#Non Lazy : It does not wait. It gets instant evaluated.
x = 1
y = 2

In [48]:
%time
f = x*x*y + y + 2
f

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 42 µs


6

In [49]:
x = 3
f

6

In [50]:
f = x*x*y + y + 2

In [51]:
f

22

In [52]:
#Lazy : Graph
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2


In [53]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [54]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
print(f.eval(session=sess))
print(sess.run(f))
sess.close()

42
42


In [55]:
# Better way
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    print(f.eval())
    print(sess.run(f))


42
42


In [56]:
# Better way
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [57]:
# Another better way

init = tf.global_variables_initializer() # prepare an init node

with tf.Session() as sess:
    init.run() # actually initialize all the variables
    print(f.eval())
    print(sess.run(f))


42
42


In [58]:
tf?

#**Managing Graphs**

In [59]:
tf.get_default_graph

<function tensorflow.python.framework.ops.get_default_graph>

In [60]:
x.graph is tf.get_default_graph

False

In [61]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [62]:
reset_graph()

x1 = tf.Variable(1)


print(x1.graph is tf.get_default_graph())

True


In [63]:
graph1 = tf.Graph()

with graph1.as_default():
    x2 = tf.Variable(2)



In [64]:
print(x2.graph is graph1)
print(x2.graph is tf.get_default_graph())

True
False


In [65]:
# To reset the default group
reset_graph()

#Lifecycle of a Node Value

In [66]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15 x is computed twice

10
15


In [67]:
with tf.Session() as s:
    z_eval, y_eval = s.run([z, y])
    print(y_eval)  # 10
    print(z_eval)  # 15 x is computed once

10
15


In [68]:
zz = tf.square(y+z)
with tf.Session() as s:
    s.run([zz, z, y])
    print(zz.eval())

625


#Linear Regression with TensorFlow : Normal Equation

In [69]:
X = np.array([[1,5/3],[1, 1/9]])
y = np.array([[12/3],[21/9]])

# (XT.X)-1  .XT   .y
xt = X.T
xtx = xt.dot(X)
ixtx =  np.linalg.inv(xtx)

theta_best = ixtx.dot(xt).dot(y)
#theta_best = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
theta_best

array([[2.21428571],
       [1.07142857]])

#Housing Linear Regression with Numpy : Normal Equation

In [70]:
housing = fetch_california_housing()
m, n = housing.data.shape 
housing.data.shape 

(20640, 8)

In [71]:
np.ones((m, 1)).shape

(20640, 1)

In [72]:
housing.data[0]

array([   8.3252    ,   41.        ,    6.98412698,    1.02380952,
        322.        ,    2.55555556,   37.88      , -122.23      ])

In [73]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] # adding bias as constant
housing_data_plus_bias.data.shape 

(20640, 9)

In [74]:
housing.target[0]

4.526

In [75]:
housing.target.shape

(20640,)

In [76]:
housing.target.reshape(-1, 1).shape

(20640, 1)

In [77]:
housing_data_plus_bias[0]

array([   1.        ,    8.3252    ,   41.        ,    6.98412698,
          1.02380952,  322.        ,    2.55555556,   37.88      ,
       -122.23      ])

In [78]:
%time
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs
[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [79]:
theta_numpy.shape

(9, 1)

In [80]:
np.matmul(housing_data_plus_bias,theta_numpy)

array([[4.13164983],
       [3.97660644],
       [3.67657094],
       ...,
       [0.17125141],
       [0.31910524],
       [0.51580363]])

#Housing Linear Regression with Tensorflow :  Normal Equation

In [81]:
reset_graph()

In [82]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

In [83]:
X.shape

TensorShape([Dimension(20640), Dimension(9)])

In [84]:
y.shape


TensorShape([Dimension(20640), Dimension(1)])

In [85]:
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)),XT),y)

In [86]:
%time
with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs


array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)